In [1]:
import pandas as pd

In [15]:
# load in normal zpids for testing
zpids_df = pd.read_csv("D:/DataSets/msc_group_project/all_property_keys.csv")

In [21]:
addresses_all = pd.read_csv("D:/DataSets/msc_group_project/addresses_all.csv")

unique_neighbourhood_df = pd.DataFrame(addresses_all['neighbourhood'].unique(), columns=['neighbourhood'])

unique_neighbourhood_df.to_csv("D:/DataSets/msc_group_project/unique_neighbourhood_names.csv", index = False)



In [19]:
# begin by loading in the zpids that had missing information in our dataset

missing_zpids = pd.read_csv("D:/DataSets/msc_group_project/missing_zpids.csv" )

missing_zpids

TypeError: read_csv() got an unexpected keyword argument 'index'

In [13]:
def check_feature(check_str, res, return_statement):
    if res is not None and check_str in res:
        if isinstance(res[check_str], dict) and return_statement in res[check_str]:
            return res[check_str][return_statement]
    return None

In [16]:
# using a for loop let's extract the information we didn't manage to retrieve the last time
import requests
import time
# create loop that makes a request for each property
# each of these lists represent a table that will be in our schema

new_row_main_details = []
new_row_addresses = []
new_row_features = [] # includes window features, associated amenities, utilities, lot features, parking, patio and porch
new_row_exterior_features = []
new_row_at_a_glance = []
new_row_fees = []
new_row_price_history = []
new_row_schools_nearby = []

iterations = len(zpids_df["zpids"])
for i in range(iterations):
    print("iteration : " + str(i) + ", zpid = " + str(zpids_df["zpids"][i]))
    
    url = "https://zillow-com1.p.rapidapi.com/property"

    querystring = {"zpid":zpids_df["zpids"][i]}

    headers = {
        "X-RapidAPI-Key": "0f64ef61f3msh8589bcbe6519721p1fd4adjsn7b6792b52973",
        "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring).json()

        if response is not None:
            # a temportary main information dictionary we create just for this iteration
            temp_main_details = {}
            temp_main_details["zpid"] = response.get("zpid")
            temp_main_details["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_main_details["buildingName"] = check_feature("building",response,"buildingName")

            temp_main_details["postingGroupName"] = response["listingProvider"]["postingGroupName"]
            temp_main_details["agentName"] = response["listingProvider"]["agentName"]
            temp_main_details["overview"] = response["description"]
            temp_main_details["url"] = response["url"]
            temp_main_details["timeOnHME"] = response["timeOnZillow"]
            temp_main_details["price"] = response["price"]
            temp_main_details["lvingAreaValue"] = response["livingAreaValue"]       
            temp_main_details["bathrooms"] = response["resoFacts"]["bathrooms"]
            temp_main_details["bathroomsHalf"] = response["resoFacts"]["bathroomsHalf"]
            temp_main_details["bathroomsFull"] = response["resoFacts"]["bathroomsFull"]
            temp_main_details["bedrooms"] = response["resoFacts"]["bedrooms"]
            temp_main_details["livingArea"] = response["resoFacts"]["livingArea"]
            temp_main_details["livingAreaValue"] = response["livingAreaValue"]
            temp_main_details["stories"] = response["resoFacts"]["stories"]
            temp_main_details["yearBuilt"] = response["resoFacts"]["yearBuilt"]
            temp_main_details["homeType"] = response["resoFacts"]["homeType"]
            temp_main_details["homeStatus"] = response["homeStatus"]
            temp_main_details["datePosted"] = response["datePosted"]
            temp_main_details["rentEstimate"] = response["rentZestimate"]

            # a temportary addresses dictionary we create just for this iteration
            temp_addresses = {}
            temp_addresses["zpid"] = response["zpid"]
            temp_addresses["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_addresses["buildingName"] = check_feature("building",response,"buildingName")

            temp_addresses["city"] = response["address"].get("city")
            temp_addresses["state"] = response["address"].get("state")
            temp_addresses["streetAddress"] = response["address"].get("streetAddress")
            temp_addresses["zipcode"] = response["address"].get("zipcode")
            temp_addresses["cityRegion"] = response["resoFacts"].get("cityRegion")

            temp_addresses["stateId"] = response.get("stateId")
            temp_addresses["cityId"] = response.get("cityId")
            temp_addresses["country"] = response.get("country")
            temp_addresses["zillowBuildingUrl"] = str("https:/zillow.com/") + str(check_feature("building",response,"bdpUrl"))

            # a temporary features information dictionary we create just for this iteration
            temp_features = {}
            count = 0
            temp_features["zpid"] = response["zpid"]
            temp_features["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_features["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("appliances"):
                for i in range(len(response["resoFacts"]["appliances"])):
                    temp_features["feature " + str(count) ] = response["resoFacts"]["appliances"][i]
                    count += 1
            if response["resoFacts"].get("lotFeatures"):
                for i in range(len(response["resoFacts"]["lotFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["lotFeatures"][i]
                    count += 1
            if response["resoFacts"].get("communityFeatures"):
                for i in range(len(response["resoFacts"]["communityFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["communityFeatures"][i]
                    count += 1
            if  response["resoFacts"].get("utilities"):
                for i in range(len(response["resoFacts"]["utilities"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["utilities"][i]
                    count += 1
            if response["resoFacts"].get("windowFeatures"):
                for i in range(len(response["resoFacts"]["windowFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["windowFeatures"][i]
                    count += 1
            if response["resoFacts"].get("laundryFeatures"):
                for i in range(len(response["resoFacts"]["laundryFeatures"])):
                    temp_features["feature" + str(count)] = "Laundry Features: " + response["resoFacts"]["laundryFeatures"][i]
                    count += 1
            if response["resoFacts"].get("patioAndPorchFeatures"):
                for i in range(len(response["resoFacts"]["patioAndPorchFeatures"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["patioAndPorchFeatures"][i]
                    count += 1
            if response["resoFacts"].get("associationAmenities"):
                for i in range(len(response["resoFacts"]["associationAmenities"])):
                    temp_features["feature" + str(count)] = response["resoFacts"]["associationAmenities"][i]
                    count += 1

            # a temporary exterior features information dictionary we create just for this iteration
            temp_exterior_features = {}
            temp_exterior_features["zpid"] = response["zpid"]
            temp_exterior_features["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_exterior_features["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("exteriorFeatures"):
                for i in range(len(response["resoFacts"]["exteriorFeatures"])):
                    temp_exterior_features["exterior feature " + str(i)] = response["resoFacts"]["exteriorFeatures"][i]

            # a temporary at a glance information dictionary we create just for this iteration
            temp_atAGlance = {}
            temp_atAGlance["zpid"] = response["zpid"]
            temp_atAGlance["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_atAGlance["buildingName"] = check_feature("building",response,"buildingName")
            if response["resoFacts"].get("atAGlanceFacts"):
                for prop in response["resoFacts"]["atAGlanceFacts"]:
                    temp_atAGlance[str(prop["factLabel"])] = str(prop["factValue"])

            # a temporary fees information dictionary we create just for this iteration
            temp_fees = {}
            temp_fees["zpid"] = response["zpid"]
            temp_fees["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_fees["buildingName"] = check_feature("building",response,"buildingName")

            if response["resoFacts"].get("feesAndDues"):
                for prop in response["resoFacts"]["feesAndDues"]:
                    temp_fees[str(prop["type"] + " fee")] = prop["fee"]

            # a temporary price history information dictionary we create just for this iteration
            temp_price_history = {}
            temp_price_history["zpid"] = response["zpid"]
            temp_price_history["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_price_history["buildingName"] = check_feature("building",response,"buildingName")
            if response.get("priceHistory"):
                for index, prop in enumerate(response["priceHistory"]):
                    temp_price_history["change_" + str(index) + "_date"] = prop["date"]
                    temp_price_history["change_" + str(index) + "_priceChangeRate"] = prop["priceChangeRate"]
                    temp_price_history["change_" + str(index) + "_postingIsRental"] = prop["postingIsRental"]
                    temp_price_history["change_" + str(index) + "_time"] = prop["time"]
                    temp_price_history["change_" + str(index) + "_event"] = prop["event"]
                    temp_price_history["change_" + str(index) + "_price"] = prop["price"]

            # a temporary school nearby information dictionary we create just for this iteration
            temp_schools_nearby = {}
            temp_schools_nearby["zpid"] = response["zpid"]
            temp_schools_nearby["geoLocation"] = str(str(response["latitude"]) + ", " + str(response["longitude"]))
            temp_schools_nearby["buildingName"] = check_feature("building",response,"buildingName")

            if response.get("schools"):
                for index, prop in enumerate(response["schools"]):
                    temp_schools_nearby["school_" + str(index) + "_name"] = prop["name"]
                    temp_schools_nearby["school_" + str(index) + "_rating"] = prop["rating"]
                    temp_schools_nearby["school_" + str(index) + "_distance"] = prop["distance"]
                    temp_schools_nearby["school_" + str(index) + "_link"] = prop["link"]
                    temp_schools_nearby["school_" + str(index) + "_level"] = prop["level"]
                    temp_schools_nearby["school_" + str(index) + "_grades"] = prop["grades"]
                    temp_schools_nearby["school_" + str(index) + "_type"] = prop["type"]
                    
            
            new_row_main_details.append(temp_main_details)
            new_row_addresses.append(temp_addresses)
            new_row_features.append(temp_features)
            new_row_exterior_features.append(temp_exterior_features)
            new_row_at_a_glance.append(temp_atAGlance)
            new_row_fees.append(temp_fees)
            new_row_price_history.append(temp_price_history)
            new_row_schools_nearby.append(temp_schools_nearby)

        else:
            # Handle the case where the JSON response is not as expected
            error_message = "Invalid JSON response"

    except Exception as e:
        # Handle any exceptions that occur during the request
        print(f"An error occurred during the request: {str(e)}")
        

    
    
        # If we have made 1 requests and it's not the last iteration, sleep for 2 seconds
    time.sleep(2)
    
main_details_df = pd.DataFrame(new_row_main_details)
address_details_df = pd.DataFrame(new_row_addresses)
indoor_features_df = pd.DataFrame(new_row_features)
exterior_features_df = pd.DataFrame(new_row_exterior_features)
at_a_glance_df = pd.DataFrame(new_row_at_a_glance)
fees_df = pd.DataFrame(new_row_fees)
price_history_df = pd.DataFrame(new_row_price_history)
schools_nearby_df = pd.DataFrame(new_row_schools_nearby)

pd.set_option('display.max_columns', None)

iteration : 0, zpid = 2075342017
iteration : 1, zpid = 2075330457


KeyboardInterrupt: 

NameError: name 'main_details_df' is not defined